<a href="https://colab.research.google.com/github/DIFACQUIM/Cursos/blob/main/4_Espacio_Qu%C3%ADmico_t_SNE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de componentes principales (PCA)


---
Realizó: Ana Chávez

Contacto: anachavez3026@gmail.com

In [ ]:
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 51.3 MB/s eta 0:00:00


In [ ]:
!pip install molplotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
Discarding https://files.pythonhosted.org/packages/39/40/c0d86942ba668d975570a0fbd7fe4224445198a90a64ec6f0c1cd3bf2527/molplotly-1.1.8.tar.gz (from https://pypi.org/simple/molplotly/): Requested molplotly from https://files.pythonhosted.org/packages/39/40/c0d86942ba668d975570a0fbd7fe4224445198a90a64ec6f0c1cd3bf2527/molplotly-1.1.8.tar.gz has inconsistent version: expected '1.1.8', but metadata has '1.1.7'
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.4 MB/s eta 0:00:00
  Created wheel for molplotly: filename=molplotly-1.1.7-py3-none-any.whl size=14048 sha25

In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import MACCSkeys, AllChem
from scipy.spatial.distance import pdist

In [ ]:
# Leer bases de datos

#BIOFACQUIM
url_biofacquim = "https://raw.githubusercontent.com/DIFACQUIM/Cursos/main/Pre-simposio%20QF/Espacio_quimico/Datasets/BIOFACQUIM.V2_curada.csv"
BIOFACQUIM = pd.read_csv(url_biofacquim)
print(BIOFACQUIM.head(2))

#FDA
url_fda = "https://raw.githubusercontent.com/DIFACQUIM/Cursos/main/Pre-simposio%20QF/Espacio_quimico/Datasets/FDA_2022_july_05_curada.csv"
FDA = pd.read_csv(url_fda)
FDA.head(2)

#DNMT1
url_dnmt1 = "https://raw.githubusercontent.com/DIFACQUIM/Cursos/main/Pre-simposio%20QF/Espacio_quimico/Datasets/DNMT1_curada.csv"
DNMT1 = pd.read_csv(url_dnmt1)
DNMT1.head(2)

        ID                                             SMILES    Data set
0  FQNP502       c1cc2c(cc1C1OCC3C(c4ccc5c(c4)OCO5)OCC13)OCO2  BIOFACQUIM
1  FQNP281  C=C(C)C(C)(C)CCC(C)C1CCC2(C)C3CCC4C(C)(C)C5CCC...  BIOFACQUIM


,ID,SMILES,Data set
0,"""CHEMBL2336409",Cc1cc(=Nc2ccc(NC(=O)c3ccc(N=c4cc[nH]c5ccccc45)...,DNMT1
1,"""CHEMBL1361703",COc1ccccc1CNC(=O)COC(=O)c1cc(-c2ccco2)nc2ccccc12,DNMT1


In [ ]:
# Ver columnas
print(FDA.columns)
print(BIOFACQUIM.columns)
print(DNMT1.columns)

Index(['ID', 'SMILES', 'NEW_SMILES', 'Data set'], dtype='object')
Index(['ID', 'SMILES', 'Data set'], dtype='object')
Index(['ID', 'SMILES', 'Data set'], dtype='object')


In [ ]:
# Seleccionar columnas
FDA = FDA[['ID', 'NEW_SMILES', "Data set"]]
BIOFACQUIM = BIOFACQUIM[['ID', 'SMILES', "Data set"]]
DNMT1 = DNMT1[['ID', 'SMILES', "Data set"]]
# Cambiar nombre a columnas
FDA.columns = ["ID", "SMILES", "Data set"]
BIOFACQUIM.columns = ['ID',  'SMILES', "Data set"]
DNMT1.columns = ["ID", "SMILES", "Data set"]
FDA.head(2)

,ID,SMILES,Data set
0,DB00006,CCC(C)C(NC(=O)C(CCC(=O)O)NC(=O)C(CCC(=O)O)NC(=...,FDA
1,DB00007,CCNC(=O)C1CCCN1C(=O)C(CCCN=C(N)N)NC(=O)C(CC(C)...,FDA


In [ ]:
# Unir (concatenar) bases de datos
data = pd.concat([FDA, BIOFACQUIM, DNMT1], axis=0).reset_index(drop=True)

In [ ]:
# Generar descriptores
# Importar bibliotecas
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors

#Calcular descriptores moleculares
data["HBA"] = [Descriptors.NumHAcceptors(y) for y in (Chem.MolFromSmiles(x) for x in data["SMILES"])]
data["HBD"] = [Descriptors.NumHDonors(y) for y in (Chem.MolFromSmiles(x) for x in data["SMILES"])]
data["RB"] = [Descriptors.NumRotatableBonds(y) for y in (Chem.MolFromSmiles(x) for x in data["SMILES"])]
data["LogP"] = [Descriptors.MolLogP(y) for y in (Chem.MolFromSmiles(x) for x in data["SMILES"])]
data["TPSA"] = [Descriptors.TPSA(y) for y in (Chem.MolFromSmiles(x) for x in data["SMILES"])]
data["MW"] = [Descriptors.MolWt(y) for y in (Chem.MolFromSmiles(x) for x in data["SMILES"])]
data.head(2)

,ID,SMILES,Data set,HBA,HBD,RB,LogP,TPSA,MW
0,DB00006,CCC(C)C(NC(=O)C(CCC(=O)O)NC(=O)C(CCC(=O)O)NC(=...,FDA,29,27,66,-8.3261,904.07,2180.317
1,DB00007,CCNC(=O)C1CCCN1C(=O)C(CCCN=C(N)N)NC(=O)C(CC(C)...,FDA,14,15,32,-1.4381,431.54,1209.421


In [ ]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
data_tsne = data.copy()
data_tsne = data_tsne.drop(labels = ["Data set", "ID","SMILES"],axis = 1)
data_tsne = StandardScaler().fit_transform(data_tsne)
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(data_tsne)
tsne_results

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 3231 samples in 0.006s...
[t-SNE] Computed neighbors for 3231 samples in 0.158s...
[t-SNE] Computed conditional probabilities for sample 1000 / 3231
[t-SNE] Computed conditional probabilities for sample 2000 / 3231
[t-SNE] Computed conditional probabilities for sample 3000 / 3231
[t-SNE] Computed conditional probabilities for sample 3231 / 3231
[t-SNE] Mean sigma: 0.199656
[t-SNE] KL divergence after 250 iterations with early exaggeration: 71.133026
[t-SNE] KL divergence after 300 iterations: 1.785170


array([[11.041617  , -0.94086605],
       [10.157844  , -0.6406828 ],
       [10.319824  , -0.5992008 ],
       ...,
       [ 4.6085362 , -7.300657  ],
       [ 4.662191  , -6.9937224 ],
       [ 1.7237955 , -5.659888  ]], dtype=float32)

In [ ]:
label = data[["Data set", "ID", "SMILES"]]
label = label.to_numpy()
label

array([['FDA', 'DB00006',
        'CCC(C)C(NC(=O)C(CCC(=O)O)NC(=O)C(CCC(=O)O)NC(=O)C(Cc1ccccc1)NC(=O)C(CC(=O)O)NC(=O)CNC(=O)C(CC(N)=O)NC(=O)CNC(=O)CNC(=O)CNC(=O)CNC(=O)C1CCCN1C(=O)C(CCCN=C(N)N)NC(=O)C1CCCN1C(=O)C(N)Cc1ccccc1)C(=O)N1CCCC1C(=O)NC(CCC(=O)O)C(=O)NC(CCC(=O)O)C(=O)NC(Cc1ccc(O)cc1)C(=O)NC(CC(C)C)C(=O)O'],
       ['FDA', 'DB00007',
        'CCNC(=O)C1CCCN1C(=O)C(CCCN=C(N)N)NC(=O)C(CC(C)C)NC(=O)C(CC(C)C)NC(=O)C(Cc1ccc(O)cc1)NC(=O)C(CO)NC(=O)C(Cc1c[nH]c2ccccc12)NC(=O)C(Cc1c[nH]cn1)NC(=O)C1CCC(=O)N1'],
       ['FDA', 'DB00014',
        'CC(C)CC(NC(=O)C(COC(C)(C)C)NC(=O)C(Cc1ccc(O)cc1)NC(=O)C(CO)NC(=O)C(Cc1c[nH]c2ccccc12)NC(=O)C(Cc1c[nH]cn1)NC(=O)C1CCC(=O)N1)C(=O)NC(CCCN=C(N)N)C(=O)N1CCCC1C(=O)NNC(N)=O'],
       ...,
       ['DNMT1', '"CHEMBL4866673',
        'COc1cc2c(=NCc3ccc(-c4ccc(C(O)=NO)cc4)cc3)cc(-c3ccc(C)o3)[nH]c2cc1OCCCN1CCCC1'],
       ['DNMT1', '"CHEMBL4878570',
        'ON=C(O)C1CCN(CC(O)COC(Cn2c3ccccc3c3ccccc32)Cn2c3ccccc3c3ccccc32)CC1'],
       ['DNMT1', '"CHEMBL4454

In [ ]:
tsne_results

array([[11.041617  , -0.94086605],
       [10.157844  , -0.6406828 ],
       [10.319824  , -0.5992008 ],
       ...,
       [ 4.6085362 , -7.300657  ],
       [ 4.662191  , -6.9937224 ],
       [ 1.7237955 , -5.659888  ]], dtype=float32)

In [ ]:
# Concatenar arrays de numpy
arr = np.concatenate((label, tsne_results), axis = 1)
arr.shape

(3231, 5)

In [ ]:
# Crear un nuevo dataframe
tsne_dataset = pd.DataFrame(data=arr, columns = ['Data set',"ID", "SMILES",'axis 1', 'axis 2'] )
tsne_dataset.head(2)

,Data set,ID,SMILES,axis 1,axis 2
0,FDA,DB00006,CCC(C)C(NC(=O)C(CCC(=O)O)NC(=O)C(CCC(=O)O)NC(=...,11.041617,-0.940866
1,FDA,DB00007,CCNC(=O)C1CCCN1C(=O)C(CCCN=C(N)N)NC(=O)C(CC(C)...,10.157844,-0.640683


In [ ]:
def color(label):
  if label == str("BIOFACQUIM"):
    return int(0)
  if label == str("FDA"):
    return int(1)
  if label == str("DNMT1"):
    return int(2)

In [ ]:
tsne_dataset["color"] = [color(x) for x in tsne_dataset["Data set"]]
tsne_dataset.head(2)

,Data set,ID,SMILES,axis 1,axis 2,color
0,FDA,DB00006,CCC(C)C(NC(=O)C(CCC(=O)O)NC(=O)C(CCC(=O)O)NC(=...,11.041617,-0.940866,1
1,FDA,DB00007,CCNC(=O)C1CCCN1C(=O)C(CCCN=C(N)N)NC(=O)C(CC(C)...,10.157844,-0.640683,1


In [ ]:
import plotly.express as px

fig = px.scatter(tsne_dataset, x="axis 1", y="axis 2", color="color",
                 color_continuous_scale=['orange', "indigo", "green"]
                 )
import molplotly
# add molecules to the plotly graph - returns a Dash app
app = molplotly.add_molecules(fig=fig,
                            df=tsne_dataset,
                            smiles_col='SMILES',
                            title_col='Data set',
                            color_col="ID"
                            )

# run Dash app inline in notebook (or in an external server)
app.run_server(mode='inline', port=8700, height=1000)

Dash is running on http://127.0.0.1:8700/



INFO:dash.dash:Dash is running on http://127.0.0.1:8700/



<IPython.core.display.Javascript object>

In [ ]:
#===============================================================================